In this experiment, we decode Shor's nine-qubit quantum error correcting code.
We show decoding of the Shor's nine-qubit code using Dephasing DMRG, which is our own built-in DMRG-like optimisation algorithm to solve the main component problem which in its turn is the problem of finding a computational basis state cotributing the most to a given state.

In [ ]:
import qecstruct as qec
from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.optimiser.utils import (
    COPY_RIGHT,
    SWAP,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    linear_css_code_checks,
    css_code_logicals,
    get_css_code_constraint_sites,
    get_css_code_logicals_sites,
    apply_parity_constraints,
    apply_bias_channel,
)

Let us first import the code from `qecstruct` and take a look at it.

In [ ]:
code = qec.shor_code()
code

This quantum error correcting code is defined on 9 physical qubits and has 2 logical operators. This means we will need $9*2 + 2 = 20$ sites in our MPS.

In [ ]:
num_sites = 2 * len(code) + code.num_x_logicals() + code.num_z_logicals()
assert num_sites == 20

Now, let us define the initial state. First of all we will check that no error will constitute no correction. This means starting from the all-zeros state followed by decoding will return all-zeros state for the logical operators (the final logical operator will be identity operator).

In [ ]:
string_state = "11" + "0" * (num_sites - 2)
error_state = create_custom_product_state(string=string_state, form="Right-canonical")

Here, we get the sites where the checks will be applied. We will need to construct MPOs using this data.

In [ ]:
checks_x, checks_z = linear_css_code_checks(code)
print("X checks:")
for check in checks_x:
    print(check)
print("Z checks:")
for check in checks_z:
    print(check)

These lists mention only the sites where we will apply the XOR constraints. However, the MPOs will also consist of other tensors, such as SWAPs (wire crossings) and boundary XOR constraints. In what follows we define the list of these auxiliary tensors and the corresponding sites where they reside.

In [ ]:
tensors_constraints = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
csscode_constraint_sites = get_css_code_constraint_sites(code)
print("Full X-check lists of sites:")
for string in csscode_constraint_sites[0]:
    print(string)
print("Full Z-check lists of sites:")
for string in csscode_constraint_sites[1]:
    print(string)

Let us now take a look at the logical operators.

In [ ]:
print(code.x_logicals_binary())
print(code.z_logicals_binary())

We need to again translate it to our MPO language by changing the indices since we add the logical-operator sites to the end of the MPS.

In [ ]:
print(css_code_logicals(code)[0])
print(css_code_logicals(code)[1])

Now goes the same operation of adding sites where auxiliary tensors go.

In [ ]:
print(get_css_code_logicals_sites(code)[0])
print(get_css_code_logicals_sites(code)[1])

In [ ]:
tensors_logicals = [XOR_LEFT, XOR_BULK, SWAP, COPY_RIGHT]
constraint_sites_logicals = get_css_code_logicals_sites(code)

Now the fun part, contracting the logical MPOs.

In [ ]:
error_state = apply_bias_channel(basis_mps=error_state, codeword_string=string_state, prob_channel=0.05)
error_state = apply_parity_constraints(
    error_state, csscode_constraint_sites[0], tensors_constraints, renormalise=True
)
error_state = apply_parity_constraints(
    error_state, csscode_constraint_sites[1], tensors_constraints, renormalise=True
)

In [ ]:
error_state = apply_parity_constraints(
    error_state, constraint_sites_logicals, tensors_logicals, renormalise=True
)

Marginalise over the message bits to get the logical operator MPS.

In [ ]:
sites_to_marginalise = list(range(2 * len(code)))
logical = marginalise(error_state, sites_to_marginalise=sites_to_marginalise)
print(logical.dense().round(3))

Which is exactly the all-zero state!